In [1]:
import sys
if "google.colab" in sys.modules:
    !wget "https://raw.githubusercontent.com/ndcbe/CBE60499/main/notebooks/helper.py"
    import helper
    helper.install_glpk ()
    helper.install_idaes()
    helper.install_ipopt()
!apt-get install -y coinor-cbc

--2025-04-23 11:10:21--  https://raw.githubusercontent.com/ndcbe/CBE60499/main/notebooks/helper.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7171 (7.0K) [text/plain]
Saving to: ‘helper.py’

helper.py           100%[===================>]   7.00K  --.-KB/s    in 0s      

2025-04-23 11:10:21 (46.5 MB/s) - ‘helper.py’ saved [7171/7171]

Installing glpk via apt-get...
Installing idaes via pip...
idaes was successfully installed
Running idaes get-extensions to install Ipopt, k_aug, and more...
Ipopt 3.13.2 (x86_64-pc-linux-gnu), ASL(20190605)

[K_AUG] 0.1.0, Part of the IDAES PSE framework
Please visit https://idaes.org/ (x86_64-pc-linux-gnu), ASL(20190605)

Couenne 0.5.8 -- an Open-Source solver for Mixed Integer Nonlinear Optimization
Mailing list: couenn

In [44]:
import pandas as pd

df_clients = pd.read_csv('/content/clients.csv')
df_depots = pd.read_csv('/content/depots.csv')
df_vehicles = pd.read_csv('/content/vehicles.csv')

In [45]:
df_depots['DepotID'] = df_depots['DepotID'].astype(str)
df_depots['ClientID'] = df_clients['ClientID'].astype(str)
df_vehicles['VehicleName'] = [f'V{i+1}' for i in range(len(df_vehicles))]


coord_depots = list(zip(df_depots['Longitude'], df_depots['Latitude']))

coord_clients = list(zip(df_clients['Longitude'], df_clients['Latitude']))

lugares = coord_depots + coord_clients

In [46]:
from pyomo.environ import *
import requests

model = ConcreteModel()

# Definición de conjuntos
model.D = Set(initialize=df_depots['DepotID'].apply(lambda x: f'CD{x}'))
model.C = Set(initialize=df_clients['ClientID'].apply(lambda x: f'C{x}'))
model.V = Set(initialize=df_vehicles['VehicleName'].tolist())
model.N = model.D | model.C


print("model.D =", list(model.D.data()))
print("model.C =", list(model.C.data()))
print("model.V =", list(model.V.data()))
print("model.N =", list(model.N.data()))
print((lugares))
print(len(lugares))

model.D = ['CD1', 'CD2', 'CD3', 'CD4', 'CD5', 'CD6', 'CD7', 'CD8', 'CD9', 'CD10', 'CD11', 'CD12']
model.C = ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9']
model.V = ['V1', 'V2', 'V3', 'V4', 'V5', 'V6']
model.N = ['CD1', 'CD2', 'CD3', 'CD4', 'CD5', 'CD6', 'CD7', 'CD8', 'CD9', 'CD10', 'CD11', 'CD12', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9']
[(-74.08124218159384, 4.75021190869025), (-74.10993358606953, 4.5363832206427785), (-74.03854814565923, 4.792925960208614), (-74.06706883098641, 4.72167778077445), (-74.13826337931849, 4.607707046760958), (-74.12400186370824, 4.650463053612691), (-74.09561875464892, 4.621911772492814), (-74.10975623736951, 4.678960680833056), (-74.09547235719887, 4.735973062153282), (-74.10991610076434, 4.550640992537941), (-74.10977422186991, 4.664702960902753), (-74.12408925943333, 4.5791740634103135), (-74.19699184741948, 4.632552840424734), (-74.15503659129905, 4.601328214582278), (-74.10178731827096, 4.732421040989568), (-74.19486224157397, 4

In [47]:
from geopy.distance import geodesic

# Datos
precio_km_gas= 3032.1
precio_km_ev= 4000
precio_km_dr= 2000


nodes = list(model.N)
print(nodes)

# Construcción de la URL
coordenadas = ";".join([f"{lon},{lat}" for lon, lat in lugares])
url = f"http://router.project-osrm.org/table/v1/driving/{coordenadas}?annotations=distance"

# Realiza la solicitud
respuesta = requests.get(url)

# Diccionarios para almacenar los resultados
cost_gas = {}
cost_ev = {}
cost_dr = {}
distance_tierra = {}
distance_aire = {}

# Verifica que la solicitud fue exitosa
if respuesta.status_code == 200:
    datos = respuesta.json()
    matriz_distancias = datos["distances"]

    for i,i1,i2 in zip(range(len(lugares)), nodes,lugares):
        for j,j1,j2 in zip(range(len(lugares)), nodes,lugares):
            if i != j:
                distancia_km_tierra = matriz_distancias[i][j] / 1000  # convertir a kilómetros
                distancia_km_aire = geodesic(i2, j2).km

                precio_gas = round(distancia_km_tierra * precio_km_gas, 3)
                precio_ev = round(distancia_km_tierra * precio_km_ev, 3)
                precio_aire = round(distancia_km_aire * precio_km_dr, 3)
            else:
                distancia_km_tierra = 9999999
                distancia_km_aire = 9999999
                precio_gas = 9999999
                precio_ev = 9999999
                precio_aire =9999999
            distance_tierra[(i1, j1)] = distancia_km_tierra
            distance_aire[(i1, j1)] = distancia_km_aire
            cost_gas[(i1, j1)] = precio_gas
            cost_ev[(i1, j1)] = precio_ev
            cost_dr[(i1, j1)] = precio_aire

print(distance_tierra)
print(distance_aire)
print(cost_gas)
print(cost_ev)
print(cost_dr)


['CD1', 'CD2', 'CD3', 'CD4', 'CD5', 'CD6', 'CD7', 'CD8', 'CD9', 'CD10', 'CD11', 'CD12', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9']
{('CD1', 'CD1'): 9999999, ('CD1', 'CD2'): 33.0612, ('CD1', 'CD3'): 10.1607, ('CD1', 'CD4'): 6.4529, ('CD1', 'CD5'): 21.415599999999998, ('CD1', 'CD6'): 16.9241, ('CD1', 'CD7'): 20.126900000000003, ('CD1', 'CD8'): 11.3612, ('CD1', 'CD9'): 4.329899999999999, ('CD1', 'CD10'): 31.2895, ('CD1', 'CD11'): 13.659, ('CD1', 'CD12'): 26.223200000000002, ('CD1', 'C1'): 22.414900000000003, ('CD1', 'C2'): 25.897, ('CD1', 'C3'): 4.2255, ('CD1', 'C4'): 22.3388, ('CD1', 'C5'): 12.0648, ('CD1', 'C6'): 18.0886, ('CD1', 'C7'): 19.57, ('CD1', 'C8'): 10.4852, ('CD1', 'C9'): 18.5532, ('CD2', 'CD1'): 32.9467, ('CD2', 'CD2'): 9999999, ('CD2', 'CD3'): 34.1802, ('CD2', 'CD4'): 29.493599999999997, ('CD2', 'CD5'): 13.9169, ('CD2', 'CD6'): 18.5269, ('CD2', 'CD7'): 14.5751, ('CD2', 'CD8'): 23.069599999999998, ('CD2', 'CD9'): 31.601599999999998, ('CD2', 'CD10'): 2.5714, ('CD2',

In [48]:
depot_capacity = {}
for depot, d in zip(df_depots['DepotID'], df_depots['Capacity']):
    depot_capacity[f'CD{depot}'] = d

print(depot_capacity)

demand = {}
for client, c in zip(df_clients['ClientID'], df_clients['Product']):
    demand[f'C{client}'] = c

print(demand)

vehicle_capacity={}
for vehicle, c in zip(df_vehicles['VehicleName'], df_vehicles['Capacity']):
    vehicle_capacity[vehicle] = c

print(vehicle_capacity)

vehicle_range={}
for vehicle, r in zip(df_vehicles['VehicleName'], df_vehicles['Range']):
    vehicle_range[vehicle] = r

print(vehicle_range)


vehicle_type={}
for vehicle, t in zip(df_vehicles['VehicleName'], df_vehicles['VehicleType']):
    vehicle_type[vehicle] = t

print(vehicle_type)

{'CD1': 8, 'CD2': 10, 'CD3': 0, 'CD4': 4, 'CD5': 28, 'CD6': 3, 'CD7': 0, 'CD8': 10, 'CD9': 43, 'CD10': 1, 'CD11': 16, 'CD12': 18}
{'C1': 12, 'C2': 15, 'C3': 15, 'C4': 6, 'C5': 5, 'C6': 11, 'C7': 12, 'C8': 10, 'C9': 15}
{'V1': 131.9211396722696, 'V2': 108.4356199315333, 'V3': 91.50425520531196, 'V4': 32.896064077536955, 'V5': 22.65262807032524, 'V6': 22.682911535937688}
{'V1': 145.85207096486445, 'V2': 1304.605971281605, 'V3': 953.172608610164, 'V4': 17.302304187458727, 'V5': 16.627680130757895, 'V6': 13.602810739289229}
{'V1': 'Gas Car', 'V2': 'EV', 'V3': 'EV', 'V4': 'drone', 'V5': 'drone', 'V6': 'drone'}


In [49]:
model.cost_gas = Param(model.N, model.N, initialize=lambda model, i, j: cost_gas[(i, j)], within=NonNegativeReals)
model.cost_ev = Param(model.N, model.N, initialize=lambda model, i, j: cost_ev[(i, j)], within=NonNegativeReals)
model.cost_dr = Param(model.N, model.N, initialize=lambda model, i, j: cost_dr[(i, j)], within=NonNegativeReals)
model.distance_tierra = Param(model.N, model.N, initialize=lambda model, i, j: distance_tierra[(i, j)], within=NonNegativeReals)
model.distance_aire = Param(model.N, model.N, initialize=lambda model, i, j: distance_aire[(i, j)], within=NonNegativeReals)
model.vehicle_type = Param(model.V, initialize=vehicle_type, within=Any)


# Variables de decisión y auxiliares

model.x = Var(model.V, model.N, model.N, domain=Binary)
model.y = Var(model.V, model.D, domain=Binary)
model.u = Var(model.V, model.C, domain=NonNegativeReals)

# funcion objetivo

def costo_vehiculo(model, v, i, j):
    tipo = model.vehicle_type[v]
    if tipo == 'Gas Car':
        return model.cost_gas[i, j]
    elif tipo == 'EV':
        return model.cost_ev[i, j]
    elif tipo == 'drone':
        return model.cost_dr[i, j]
    return 9999999  # penalización grande si no reconoce el tipo

model.costo = Expression(model.V, model.N, model.N, rule=costo_vehiculo)

def distancia_vehiculo(model, v, i, j):
    tipo = model.vehicle_type[v]
    if tipo == 'drone':
        return model.distance_aire[i, j]
    else:
        return model.distance_tierra[i, j]

model.distancia = Expression(model.V, model.N, model.N, rule=distancia_vehiculo)


def obj_rule(model):
    return sum(model.costo[v, i, j] * model.x[v, i, j] for v in model.V for i in model.N for j in model.N if i != j)

model.obj = Objective(rule=obj_rule, sense=minimize)

# Restricciones

def assign_depot_rule(model, v):
    return sum(model.y[v,d] for d in model.D) == 1

model.assign_depot = Constraint(model.V, rule=assign_depot_rule)
"""
def depot_uniqueness_rule(model, d):
    return sum(model.y[v,d] for v in model.V) <= 1

model.depot_uniqueness = Constraint(model.D, rule=depot_uniqueness_rule)
"""
def start_route_rule(model, v, d):
    return sum(model.x[v, d, j] for j in model.N if j != d) == model.y[v,d]

model.start_route = Constraint(model.V, model.D, rule=start_route_rule)

def return_route_rule(model, v, d):
    return sum(model.x[v, i, d] for i in model.N if i != d) == model.y[v,d]

model.return_route = Constraint(model.V, model.D, rule=return_route_rule)

def flow_conservation_rule(model, v, i):
    if i in model.C:
        return sum(model.x[v, j, i] for j in model.N if j != i) == \
               sum(model.x[v, i, j] for j in model.N if j != i)
    else:
        return Constraint.Skip

model.flow_conservation = Constraint(model.V, model.N, rule=flow_conservation_rule)

def no_depot_to_depot_rule(model, v, i, j):
    if i in model.D and j in model.D and i != j:
        return model.x[v,i,j] == 0
    else:
        return Constraint.Skip

model.no_depot_travel = Constraint(model.V, model.N, model.N, rule=no_depot_to_depot_rule)

def customer_visit_rule(model, i):
    if i in model.C:
        return sum(model.x[v, j, i] for v in model.V for j in model.N if j != i) == 1
    else:
        return Constraint.Skip

model.customer_visit = Constraint(model.N, rule=customer_visit_rule)

def vehicle_capacity_rule(model, v):
    return sum(demand[i] * sum(model.x[v, j, i] for j in model.N if j != i) for i in model.C) <= vehicle_capacity[v]

model.vehicle_capacity = Constraint(model.V, rule=vehicle_capacity_rule)

def depot_capacity_rule(model, v):
    return sum(demand[i] * sum(model.x[v, j, i] for j in model.N if j != i) for i in model.C) <= sum(model.y[v,d] * depot_capacity[d] for d in model.D)

model.depot_capacity = Constraint(model.V, rule=depot_capacity_rule)

def vehicle_range_rule(model, v):
    return sum(model.distancia[v, i, j] * model.x[v, i, j] for i in model.N for j in model.N if i != j) <= vehicle_range[v]

model.vehicle_range = Constraint(model.V, rule=vehicle_range_rule)


def mtz_rule(model, v, i, j):
    if i != j:
        return model.u[v,i] - model.u[v,j] + vehicle_capacity[v] * model.x[v,i,j] <= vehicle_capacity[v] - demand[j]
    return Constraint.Skip

model.mtz = Constraint(model.V, model.C, model.C, rule=mtz_rule)

def mtz_bounds_rule(model, v, i):
    return (demand[i], model.u[v,i], vehicle_capacity[v])

model.mtz_bounds = Constraint(model.V, model.C, rule=mtz_bounds_rule)

def demand_satisfaction_rule(model, v, i):
    return model.u[v,i] >= demand[i] * sum(model.x[v, j, i] for j in model.N if j != i)

model.demand_satisfaction = Constraint(model.V, model.C, rule=demand_satisfaction_rule)

# Resolver
solver = SolverFactory('cbc')
solver.options["seconds"] = 300  # máximo 5 minutos
results = solver.solve(model)
#results.write()

from pyomo.environ import value

for v in model.V:
    # Encuentra el depósito asignado al vehículo v
    deposito_asignado = None
    for d in model.D:
        if value(model.y[v, d]) >= 0.5:
            deposito_asignado = d
            break

    if deposito_asignado is None:
        print(f"Vehículo {v}: No se le asignó un depósito.")
        continue

    # Construir ruta a partir del depósito asignado
    ruta = [deposito_asignado]
    visitados = set(ruta)
    nodo_actual = deposito_asignado

    while True:
        siguiente = None
        for j in model.N:
            if j != nodo_actual and value(model.x[v, nodo_actual, j]) >= 0.5:
                siguiente = j
                break

        if siguiente is None or siguiente == deposito_asignado:
            break

        ruta.append(siguiente)
        visitados.add(siguiente)
        nodo_actual = siguiente

    ruta.append(deposito_asignado)

    # Imprimir la ruta
    print(f"Vehículo {v}: " + " -> ".join(ruta))


Vehículo V1: CD11 -> C9 -> CD11
Vehículo V2: CD5 -> C2 -> CD5
Vehículo V3: CD9 -> C3 -> CD9
Vehículo V4: CD9 -> C8 -> C7 -> CD9
Vehículo V5: CD5 -> C1 -> C4 -> CD5
Vehículo V6: CD11 -> C5 -> C6 -> CD11


In [50]:
rutas = {}

for v in model.V:
    ruta = []

    # Buscar bodega asignada
    assigned_depot = [d for d in model.D if value(model.y[v,d]) > 0.5]
    if assigned_depot:
        origen = assigned_depot[0]
        ruta.append(origen)

        # Armar la secuencia de visitas
        actual = origen
        while True:
            next_node = None
            for j in model.N:
                if actual != j and value(model.x[v, actual, j]) > 0.5:
                    next_node = j
                    break
            if next_node is None or next_node == origen:
                ruta.append(origen)
                break
            ruta.append(next_node)
            actual = next_node

    rutas[v] = ruta

In [71]:
import requests
import folium

# Coordenadas de cada lugar (CD1, C1, C2, ..., C24)
l = {}

for i,j in zip(nodes,lugares):
    l[i] = j
print(l)
# Crear el mapa centrado en el depósito
mapa = folium.Map(location=[4.7110, -74.0721], zoom_start=13)

# Paleta de colores para distinguir rutas por vehículo
colores = [
    "blue", "red", "green", "purple", "orange", "darkred",
    "lightblue", "cadetblue", "darkgreen", "black"
]

# Dibujar rutas por vehículo
for idx, (vehiculo, puntos) in enumerate(rutas.items()):

  if vehicle_type[vehiculo] != 'drone':
    ruta_coords = []

    for i in range(len(puntos) - 1):
        origen = l[puntos[i]]
        destino = l[puntos[i + 1]]

        # Consulta a OSRM
        url = f"http://router.project-osrm.org/route/v1/driving/{origen[0]},{origen[1]};{destino[0]},{destino[1]}?overview=full&geometries=geojson"
        response = requests.get(url)
        data = response.json()

        if "routes" in data and len(data["routes"]) > 0:
            ruta = data["routes"][0]["geometry"]["coordinates"]
            ruta_folium = [(lat, lon) for lon, lat in ruta]
            ruta_coords.extend(ruta_folium)

    # Dibujar la línea completa de la ruta
    folium.PolyLine(
        ruta_coords,
        color=colores[idx % len(colores)],
        weight=4,
        opacity=0.8,
        popup=vehiculo
    ).add_to(mapa)

# Agregar marcadores de todos los nodos
for nombre, coord in l.items():
    folium.Marker(
        [coord[1], coord[0]],
        popup=nombre,
        icon=folium.Icon(color="orange" if "C" in nombre else "purple")
    ).add_to(mapa)

# Mostrar el mapa
mapa


{'CD1': (-74.08124218159384, 4.75021190869025), 'CD2': (-74.10993358606953, 4.5363832206427785), 'CD3': (-74.03854814565923, 4.792925960208614), 'CD4': (-74.06706883098641, 4.72167778077445), 'CD5': (-74.13826337931849, 4.607707046760958), 'CD6': (-74.12400186370824, 4.650463053612691), 'CD7': (-74.09561875464892, 4.621911772492814), 'CD8': (-74.10975623736951, 4.678960680833056), 'CD9': (-74.09547235719887, 4.735973062153282), 'CD10': (-74.10991610076434, 4.550640992537941), 'CD11': (-74.10977422186991, 4.664702960902753), 'CD12': (-74.12408925943333, 4.5791740634103135), 'C1': (-74.19699184741948, 4.632552840424734), 'C2': (-74.15503659129905, 4.601328214582278), 'C3': (-74.10178731827096, 4.732421040989568), 'C4': (-74.19486224157397, 4.638612189685843), 'C5': (-74.11027242131048, 4.727691608175209), 'C6': (-74.15228946899238, 4.665002925306577), 'C7': (-74.03241075805882, 4.6771016997028205), 'C8': (-74.0624758742746, 4.707007012114083), 'C9': (-74.09804168568287, 4.636074548012315